언어 판별기 딥러닝



검증까지 ......

#### [1] 모듈로딩 <hr>

In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader 
import torch.optim as optim    
from torchmetrics.classification import MulticlassF1Score
from torchinfo import summary 
import math
from collections import *

# 파일 관련
import os

#- Data 및 시각화 관련
import pandas as pd 
import matplotlib.pyplot as plt              
from sklearn.preprocessing import * 
from sklearn.model_selection import train_test_split

언어별로 사용하는 알파벳의 빈도가 다르다. 그 빈도수를 통해 딥러닝을 시켜서 어떤 언어인지 파악하도록 한다.

.....

split

In [26]:
TRAIN_PATH = r'C:\Git\KDT\BigData\data\lang\train\\'
TEST_PATH = r'C:\Git\KDT\BigData\data\lang\test\\'

files = os.listdir(TRAIN_PATH)
featureDF = pd.DataFrame()
targetSR = []

for file in files:
    label = file[:2]    # 언어 라벨

    with open(TRAIN_PATH+file, mode = 'r') as f:
        alldata = f.read()
        alldata = alldata.lower()

        alldata = alldata.replace('\n','')

        for ch in alldata:
            if ord('a') >= ord(ch) or ord(ch) <= ord('z'):
                alldata = alldata.replace(ch,'')

    count = Counter(alldata)
    countSR = pd.Series(count)
    countSR = countSR/countSR.sum()
    featureDF[file] = countSR
    targetSR.append(label)

In [22]:
count

Counter({'|': 91, 'á': 8, 'í': 4, 'ó': 1, 'é': 4})

In [20]:
featureDF

,en-1.txt,en-2.txt,en-3.txt,en-4.txt,en-5.txt,fr-10.txt,fr-6.txt,fr-7.txt,fr-8.txt,fr-9.txt,id-11.txt,id-12.txt,id-13.txt,id-14.txt,id-15.txt,tl-16.txt,tl-17.txt,tl-18.txt,tl-19.txt,tl-20.txt
,0.153846,0.428571,0.608696,0.671875,0.314286,0.083439,0.127972,0.359888,0.200855,0.139706,0.769231,0.647059,0.365591,0.413793,0.611814,0.089109,0.281553,0.025316,0.191489,NaN
°,0.307692,NaN,NaN,NaN,0.228571,0.002528,0.001399,NaN,NaN,0.003676,NaN,NaN,0.043011,NaN,0.008439,NaN,NaN,NaN,0.095745,NaN
′,0.153846,NaN,NaN,NaN,0.114286,0.002528,0.001399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002110,NaN,NaN,NaN,NaN,NaN
″,0.153846,NaN,NaN,NaN,0.114286,0.002528,0.001399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
﻿,0.230769,NaN,NaN,NaN,0.171429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003165,NaN,NaN,NaN,NaN,NaN
